In [ ]:
import train

train.train('mamba', )

In [6]:
import configs.common as cc
import configs.mamba as cm
import models
import math
config_values = cm.config.model_values
config_values.vocab_size = cc.vocab_size
config_values.d_inner = int(config_values.expand * config_values.d_model)
config_values.dt_rank = math.ceil(config_values.d_model / 16)
    
if config_values.vocab_size % config_values.pad_vocab_size_multiple != 0:
    config_values.vocab_size += (config_values.pad_vocab_size_multiple - config_values.vocab_size % config_values.pad_vocab_size_multiple)

model = models.mamba.Mamba(config_values)


In [ ]:
import train
import torch
model_type = 'mamba'
model = train.new_model(model_type)

train.train(model)

# torch.save(model.state_dict(), f'models/{model_type}/{new_name}')

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\GitHub\\dataset\\midi_dataset\\metadata.json'